In [2]:
import joblib
import pandas as pd
import numpy as np


anime_encoder = joblib.load("/kaggle/input/anime/anime_encoder.pkl")
user_encoder = joblib.load("/kaggle/input/anime/user_encoder.pkl")
rating_scaler = joblib.load("/kaggle/input/anime/scaler.pkl")

In [3]:
df_anime = pd.read_csv('/kaggle/input/myanimelist-dataset/anime-dataset-2023.csv')
df_anime.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [4]:
# After loading df_anime:
df_anime = df_anime[df_anime['anime_id'].isin(anime_encoder.classes_)]

In [5]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/input/anime/full_anime_model.keras")

In [6]:
def extract_weights(name, model):
    # Get the layer by name from the model
    weight_layer = model.get_layer(name)
    
    # Get the weights from the layer
    weights = weight_layer.get_weights()[0]
    
    # Normalize the weights
    weights = weights / np.linalg.norm(weights, axis=1).reshape((-1, 1))
    
    return weights

# Extract weights for anime embeddings
anime_weights = extract_weights('anime_embedding', model)
# Extract weights for user embeddings
user_weights = extract_weights('user_embedding', model)

In [7]:
def find_similar_animes(name, n=10, return_dist=False, neg=False):
    try:
        # Check if anime exists in df_anime
        anime_row = df_anime[df_anime['Name'] == name]
        if anime_row.empty:
            print(f'"{name}" not found in the anime list.')
            return pd.DataFrame()
        
        anime_id = anime_row.iloc[0]['anime_id']
        
        # Check if anime_id is known to the encoder
        if anime_id not in anime_encoder.classes_:
            print(f'"{name}" (ID: {anime_id}) was not in the training data. Cannot recommend.')
            return pd.DataFrame()
        
        encoded_index = anime_encoder.transform([anime_id])[0]
        weights = anime_weights
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        n = n + 1            
        
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]
        
        print(f'Animes closest to {name}')
        
        SimilarityArr = []
        for close in closest:
            decoded_id = anime_encoder.inverse_transform([close])[0]
            anime_frame = df_anime[df_anime['anime_id'] == decoded_id]
            
            if not anime_frame.empty:
                anime_name = anime_frame['Name'].values[0]
                english_name = anime_frame['English name'].values[0]
                display_name = english_name if english_name != "UNKNOWN" else anime_name
                genre = anime_frame['Genres'].values[0]
                synopsis = anime_frame['Synopsis'].values[0]
                similarity = "{:.2f}%".format(dists[close] * 100)
                SimilarityArr.append({
                    "Name": display_name,
                    "Similarity": similarity,
                    "Genres": genre,
                    "Synopsis": synopsis
                })
        
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="Similarity", ascending=False)
        return Frame[Frame.Name != name]
    
    except Exception as e:
        print(f'Error: {str(e)}')
        return pd.DataFrame()

In [8]:
synopsis_embeddings = np.load("/kaggle/input/anime-synopsis/synopsis_embeddings.npy")

In [13]:
find_similar_animes("Cowboy Bebop")

Animes closest to Cowboy Bebop


,Name,Similarity,Genres,Synopsis
9,Cowboy Bebop: The Movie,72.36%,"Action, Sci-Fi","Another day, another bounty—such is the life o..."
8,Samurai Champloo,58.22%,"Action, Adventure, Comedy",Fuu Kasumi is a young and clumsy waitress who ...
7,Mushi-Shi,55.98%,"Adventure, Mystery, Slice of Life, Supernatural","""Mushi"": the most basic forms of life in the w..."
6,FLCL,55.88%,"Action, Avant Garde, Comedy, Sci-Fi",Naota Nandaba is an ordinary sixth grader livi...
5,Ghost in the Shell: Stand Alone Complex,55.66%,"Action, Award Winning, Sci-Fi","In the not so distant future, mankind has adva..."
4,Princess Mononoke,55.01%,"Action, Adventure, Award Winning, Fantasy",When an Emishi village is attacked by a fierce...
3,The Legend of Hei,54.28%,"Adventure, Drama, Fantasy",Luo Xiaohei is a young monster who normally ta...
2,Spirited Away,54.16%,"Adventure, Award Winning, Supernatural","Stubborn, spoiled, and naïve, 10-year-old Chih..."
1,Trigun,53.99%,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0..."
0,Ghost in the Shell,53.55%,"Action, Award Winning, Mystery, Sci-Fi, Suspense","In the year 2029, Niihama City has become a te..."
